### BAX 422 Homework 1

- Willa Yu
- Iris Wan
- Mandy Gu
- John Elmer Loretizo

__Import packages needed__

In [1]:
import csv
import re
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/qimenggu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Regular Expressions
__Ingest `mock_data.csv` as a string file__

In [2]:
line = ''
with open('mock_data.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            for c in range(len(row)):
                  line += row[c] + ' '
            line_count += 1
        line += '\n'
    print(f'Processed {line_count} lines.')

Column names are birthday, phone, names, lat_long, email, zip, city
Processed 101 lines.


__Preview data__

In [3]:
print(line)


05.07.2020 1-560-294-4480 Leslie H. Howard -80.7931, 157.73725 magna.Nam@Praesentinterdumligula.com 05-449 Campina Grande 
06.11.2019 1-616-403-7121 Kyra T. Wynn -54.65661, 87.93458 amet.ornare@uterosnon.net 1975 Bagh 
23.11.2020 1-258-160-9496 Darius G. Huff -56.23283, -116.1583 ipsum@erat.co.uk 24192 Conca Casale 
07.05.2020 1-129-425-6705 Flynn M. Rodriguez 50.02477, -93.02564 nunc.est.mollis@auctor.com 6311 Uyo 
25.09.2020 1-715-932-5611 Damon K. Potts 34.94064, 91.87029 neque.venenatis@Phasellus.ca 26531 Lahore 
30.12.2019 1-488-594-4480 Odessa U. Stewart 41.01394, 96.36819 Proin.mi@a.co.uk 40738 Namen 
04.03.2019 1-693-871-0017 Ruby M. Noble 75.21109, -148.67016 gravida.non.sollicitudin@odioNam.net 968589 Sevilla 
17.12.2020 1-337-498-7475 Hadassah W. Dale 32.72148, -122.01392 Cras@dictum.co.uk 5375 Bella 
24.02.2019 1-861-878-4813 Baker B. Francis 36.06121, -68.27199 erat@tinciduntnibhPhasellus.ca 65737 Chekhov 
25.09.2020 1-825-185-8476 Cheyenne K. Dean 56.5022, 151.02082 in@n

__Question 1__ Transform the column birthday from European to US data format while leaving the rest as is.

Existing European birthday format is as follows:
$$dd.mm.YYYY$$

The desired US date format is:
$$mm/dd \ YYYY$$

Regular Expression Search Format:
`'([012][\d]|3[0-1])\.(0[1-9]|1[0-2])\.([012][\d]{3})'`

The regular expression search string will be composed of three individual capture groups separated by periods. 

__First Capture Group__ <br>
Ground truth dictates that birthdates can only be between 01 - 31. <br>
`(0[1-9]|[12][\d]|3[0-1])` : <br>
Case 1 - 0 as first digit and numbers 1 to 9 as second digit. <br>
Case 2 - 1 or 2 as first digit and any number as second digit. <br>
Case 3 - 3 as first digit and only numbers 0 and 1 as second digit.

__Second Capture Group__ <br>
Ground truth dictates that month numbers can only be between 01 - 12. <br>
`(0[1-9]|1[0-2])` : <br>
Case 1 - 0 as first digit and numbers 0 to 9 as second digit. <br>
Case 2 - 1 as first digit and only numbers 0 to 2 as second digit. 

__Third Capture Group__ <br>
Ground truth dictates that a year can only be up to 2000s. <br>
`([012][\d]{3})` : <br>
Case 1 - First digit can only be 0 - 2 and any three digits accompanying after.

Replace Format:
`'\\2/\\1 \\3'`

The replace format moves the month first followed by a forward slash and then the date and the year after the space.

In [4]:
q1 = re.sub(r'([012][\d]|3[0-1])\.(0[1-9]|1[0-2])\.([012][\d]{3})', '\\2/\\1 \\3', line)
print (q1)


07/05 2020 1-560-294-4480 Leslie H. Howard -80.7931, 157.73725 magna.Nam@Praesentinterdumligula.com 05-449 Campina Grande 
11/06 2019 1-616-403-7121 Kyra T. Wynn -54.65661, 87.93458 amet.ornare@uterosnon.net 1975 Bagh 
11/23 2020 1-258-160-9496 Darius G. Huff -56.23283, -116.1583 ipsum@erat.co.uk 24192 Conca Casale 
05/07 2020 1-129-425-6705 Flynn M. Rodriguez 50.02477, -93.02564 nunc.est.mollis@auctor.com 6311 Uyo 
09/25 2020 1-715-932-5611 Damon K. Potts 34.94064, 91.87029 neque.venenatis@Phasellus.ca 26531 Lahore 
12/30 2019 1-488-594-4480 Odessa U. Stewart 41.01394, 96.36819 Proin.mi@a.co.uk 40738 Namen 
03/04 2019 1-693-871-0017 Ruby M. Noble 75.21109, -148.67016 gravida.non.sollicitudin@odioNam.net 968589 Sevilla 
12/17 2020 1-337-498-7475 Hadassah W. Dale 32.72148, -122.01392 Cras@dictum.co.uk 5375 Bella 
02/24 2019 1-861-878-4813 Baker B. Francis 36.06121, -68.27199 erat@tinciduntnibhPhasellus.ca 65737 Chekhov 
09/25 2020 1-825-185-8476 Cheyenne K. Dean 56.5022, 151.02082 in@n

__Question 2__ Strip everything BUT the email column

Regular Expression Search Format: `[!#$%&\'*+-/=?^_'`` `{|}~\.a-zA-Z0-9]+@[-a-zA-Z0-9]+\.[-a-zA-Z0-9]+`

The capture group is generally separted between local part and the domain part. As a standard, RFC 5322 (sections 3.2.3 and 3.4.1) and RFC 5321 were followed in building the regex search format above.

In [5]:
q2 = print(re.sub(r"([0-9.].*?)([!#$%&'*+-/=?^_'\`{|}~\.\w]+@[-\w]+\.[-\w]+)([-\w, ]*)",r"\2",line))
print(q2)


magna.Nam@Praesentinterdumligula.com
amet.ornare@uterosnon.net
ipsum@erat.co.uk 24192 Conca Casale 
nunc.est.mollis@auctor.com
neque.venenatis@Phasellus.ca
Proin.mi@a.co.uk 40738 Namen 
gravida.non.sollicitudin@odioNam.net
Cras@dictum.co.uk 5375 Bella 
erat@tinciduntnibhPhasellus.ca
in@nequeseddictum.co.uk 302379 Stary Oskol 
vel@mollis.ca
egestas@mauriserat.com
egestas.hendrerit.neque@erat.net
nulla@scelerisquesed.co.uk Z44 3OC Alès 
tristique.pellentesque@egetvarius.net
erat.Etiam@senectus.com
amet.dapibus@dignissimlacusAliquam.edu
cursus.luctus@risusatfringilla.org
ut@fermentummetusAenean.com
rutrum@metusAliquam.co.uk 6751 DI Campbeltown 
Proin.dolor@sagittisDuisgravida.ca
elit.pretium.et@vitaesodales.edu
magna.a.tortor@id.net
ornare.elit.elit@adipiscingenim.org
nulla.Integer@maurisipsumporta.ca. Anne 
semper.rutrum.Fusce@Nullam.net
conubia.nostra.per@Nullamscelerisqueneque.edu
nec.eleifend.non@enim.co.uk 62623-64171 Seevetal 
nibh@Nullamsuscipit.net
elit.elit@sedfacilisis.ca
non.c

__Question 3__ Convert all rows to "name [TAB] birthday" only (and strip the rest).  The birthday column should be in the US format. ([TAB]s will allow you to copy and pass its result into excel.)


In [6]:
q3 = re.sub(r"([0-9]{2})\.([0-9]{2})\.([1,2][9,0][0-9]{2}) ([-0-9]*) ([\w .]*) (.*)",r'\5\t\2/\1 \3',line)
print(q3)


Leslie H. Howard	07/05 2020
Kyra T. Wynn	11/06 2019
Darius G. Huff	11/23 2020
Flynn M. Rodriguez	05/07 2020
Damon K. Potts	09/25 2020
Odessa U. Stewart	12/30 2019
Ruby M. Noble	03/04 2019
Hadassah W. Dale	12/17 2020
Baker B. Francis	02/24 2019
Cheyenne K. Dean	09/25 2020
Kevyn E. Cox	07/23 2019
Gregory P. Hatfield	10/05 2019
Kylee J. Hoover	03/29 2019
Flavia M. Pitts	03/23 2019
Tamara V. Patrick	12/02 2020
Byron T. Horne	02/29 2020
Aubrey O. Barr	07/07 2020
India M. Burton	08/08 2020
Tatiana O. Sloan	06/26 2020
Kirsten S. Wolf	06/11 2020
Knox C. Farmer	01/28 2019
Kieran P. Hendrix	09/06 2019
Libby F. Mcmahon	06/06 2019
Xanthus P. Long	12/05 2020
Forrest Q. Stevens	01/09 2019
Gisela M. Banks	08/17 2020
Demetrius H. Solomon	06/10 2020
Rhoda V. Thornton	10/06 2020
William N. Thompson	11/20 2020
Nathaniel R. Meadows	04/27 2020
Kerry Y. Holman	01/21 2019
Brooke Y. Jensen	09/09 2019
Carl Y. Hart	05/09 2019
Yoko W. Conley	10/12 2020
Gisela E. Zamora	10/29 2020
Emily S. Morales	12/19 2020
Ree

__Question 4__ Strip everything BUT lat_long AND reorder its entries to be "long [TAB] lat".  ([TAB]s will allow you to copy and pass its result into excel.)


In [7]:
q4 = re.sub(r'(.*?)([-.0-9]*), ([-.0-9]*)(.*)', r'\3\t\2', line)
print(q4)


157.73725	-80.7931
87.93458	-54.65661
-116.1583	-56.23283
-93.02564	50.02477
91.87029	34.94064
96.36819	41.01394
-148.67016	75.21109
-122.01392	32.72148
-68.27199	36.06121
151.02082	56.5022
46.41947	89.15273
33.01087	-59.95322
143.00006	68.95134
134.95122	-84.49028
63.55341	39.99846
100.66053	32.49449
9.15288	-50.38051
156.80774	-62.94522
24.95209	28.70745
-118.19839	-64.38445
85.92271	2.41655
33.43453	55.02022
0.14016	-39.34568
-131.54363	58.72796
-0.57186	89.20785
62.71378	14.21796
-142.31014	63.39257
-160.4698	12.23557
134.59113	-35.82526
-111.38604	-21.19925
-167.69919	-30.41505
-172.71433	25.17997
166.6208	-81.05365
29.5851	12.56792
-80.47218	-17.71338
30.07917	6.50359
119.03856	-68.94074
88.84348	-64.69129
165.22393	-69.0148
-40.28233	-66.76159
31.62998	25.43639
51.18031	70.34742
-96.72922	25.1437
-53.5339	37.48213
138.33781	70.36436
-54.80389	-15.88582
-102.64553	-19.95651
-120.88989	-59.46452
-38.76282	28.8487
3.49884	87.66421
-70.91566	-38.86361
-88.76742	39.79327
46.38781	46

### Web Scraping

For this part of the assignment, for each line of code in your submission, you are required to add a comment line prior to it that explains what that line of code is doing. Please follow these instructions:

- Go to https://www.usnews.com/

- Find its current "Top Stories" (do not hard-code its URL)

- Read + print the URL of the _second_ current top story to the screen

- Navigate to it (a.k.a the _second_ current top story)

- Read + print the header as well as the first 3 sentences of the main body to the screen

__Define the URL__

In [8]:
url = 'https://www.usnews.com/'

__Define header__

In [9]:
headers = {'User-Agent': 'Mozilla/5.0'}

__Get content from URL__

In [10]:
page = requests.get(url, headers = headers)

__Obtain a clean version of content by using BeautifulSoup__

In [11]:
soup = BeautifulSoup(page.text, 'html.parser')

__Scrape the Top Stories Section__

In [12]:
top_stories = str(soup.find(class_ = 's85n6m5-0-Box-cwadsP kdRAHx'))

__Put all stories into a list, where index shows its recency__

In [13]:
story_list = re.split(r'<div class="ArmRestTopStories__CollapseBorderContentBox-s1vkad-0 kkieLV s85n6m5-0-Box-cwadsP fHRMIQ" spacing="3">', top_stories)

__Find the first and second story__

In [14]:
first = story_list[1]
second = story_list[2]

__Return the header of the first story by matching a pattern of `<a></a>`__

In [15]:
story1_head = re.findall(r'<a href.*?</a>', first)[0].split('>')[1][:-3].strip()
story1_head

'Trudeau: Jet Shot Down by Iranian Missile'

__Get the url of the second story__

In [16]:
new_url = re.findall(r'<a href.*?</a>', second)[0].split('"')[1]
new_url

'https://www.usnews.com/news/healthiest-communities/articles/2020-01-09/study-finds-racial-disparities-with-breast-cancer-diagnosis'

__Get the content of new url and put in BeautifulSoup Format__

In [17]:
new_page = requests.get(new_url, headers = headers)
new_soup = BeautifulSoup(new_page.text, 'html.parser')

__Find the header of the story__

In [18]:
header = str(new_soup.find(class_ = 'sc-bdVaJa kwJVtY')).split('>')[1][:-4]
header

'Study Finds Racial Disparities With Breast Cancer Diagnosis'

__Get all paragraphs of the story__

In [27]:
prg_list = re.findall(r'<p>.*?</p>', str(new_soup))
# Combine all paragraphs together into one string
prg = ''
for p in prg_list:
    # Find the paragrphs by splitting the content with '<>'
    for i in re.split(r'<.*?>', p):
        if i != '':
            # Combine all paragraphs together
            prg += i + ' '

__Extract the first three sentences__
Find the first three sentences by tokenizing sentences

In [28]:
stc_3 = ' '.join(sent_tokenize(prg)[:3])
print (stc_3)

Racial and ethnic  minority women are more likely than white women to be diagnosed with breast cancer at a later stage, according to a study released Thursday. The  JAMA Oncology  study, which included more than 177,000 women in its sample, found that higher percentages of black, American Indian or Alaskan Native and Hispanic women were diagnosed with stage 3 breast cancer than white and Asian or Pacific Islander women. All four racial and ethnic minorities studied were diagnosed with stage 2 breast cancer at higher levels than white women as well.
